## Get waze alerts history and open events from bigquery database

In [1]:
cd ../../../../Apps/Python/bolsao-api

C:\Users\luisr\Desktop\Repositories\Apps\Python\bolsao-api


In [14]:
# Python modules
import requests, json, time, pandas as pd, numpy as np, urllib
from datetime import datetime as dt
from google.cloud import bigquery
from google.oauth2 import service_account

# google cloud auth
project_id = 'pluvia-360323'
google_credentials = 'credentials/pluvia-iduff.json'
google_credentials = service_account.Credentials.from_service_account_file(google_credentials)

fields_bq = [
    'city', 'confidence', 'nThumbsUp',
    'street', 'uuid', 'country',
    'type', 'subtype', 'roadType',
    'reliability', 'magvar', 'reportRating',
    'ts', 'geo', 'geoWKT',
]

# Waze partner access token
waze_url = 'https://www.waze.com/partnerhub-api/waze-feed-access-token/c37c11ba-ff9d-4ad5-8ecc-4e4f12e91efb?format=1'

# ---
# Waze partner feed methods

def get_waze_partner_alerts(alert_type='alerts', datetime_as_str=True):
    time_cols = ['pubMillis', 'detectionDateMillis', 'updateDateMillis']
    "alert_type: one of 'alerts', 'irregularities', 'jams'"
    incidents = requests.get(waze_url).json()
    if alert_type not in incidents.keys():
        return None
    df = pd.DataFrame(incidents[alert_type])
    # Data cleaning & preprocessing
    if alert_type == 'alerts':
        df[['longitude', 'latitude']] = list(df['location'].map(lambda coords: [coords['x'], coords['y']]))
        df.drop('location', axis=1, inplace=True)        
    for col in time_cols:
        if col in df.columns:
            df[col] = (df[col] / 1000).map(dt.fromtimestamp)
            df.sort_values(col, ascending=False, inplace=True)
            if datetime_as_str: # Convert datetime column to string
                df[col] = df[col].astype(str)
    df.reset_index(drop=True, inplace=True)
    return df.fillna(np.nan).replace([np.nan], [None])

# ---
# BIGQUERY DATASET METHODS

def bigquery_request(query):
    client = bigquery.Client(credentials=google_credentials)
    query_job = client.query(query)
    return pd.DataFrame(list(map(dict, query_job.result())))

def where_clause(items):
    return ' AND '.join([' '.join([key, 'IN', f"""({', '.join([f'"{value}"' for value in (values if type(values) is list else values.split(','))])})"""]) for key, values in items.items()])

def get_waze_bq_history(
    start=dt(2023,1,1), end='now',
    where={}, select='*', as_datetime=False,
):
    waze_query = '''
    SELECT {} FROM `waze-public-dataset.partner_EscritriodeDadosdoRiodeJaneiro.view_alerts_clustered` WHERE ts BETWEEN "{}" AND "{}"
    '''
    if end == 'now': end = dt.now()
    if type(select) is list: select = ', '.join(select)
    query = waze_query.format(select, str(start), str(end))
    if len(where):
        query = ' '.join([query, 'AND', where_clause(where)])
    history = bigquery_request(query)
    if len(history):
        if not as_datetime:
            for col in ['ts']:
                if col in history.columns:
                    history[col] = history[col].map(lambda stamp: stamp.strftime('%Y-%m-%d %X'))
        return history
    else:
        return pd.DataFrame(columns=fields_bq)

def get_waze_bq_open_events(
    at=dt.now(), where={}, select='*',
    between=7*pd.offsets.Day(), as_datetime=False,
):
    waze_query = '''
        SELECT {} FROM `waze-public-dataset.partner_EscritriodeDadosdoRiodeJaneiro.view_alerts_clustered`
        WHERE ts BETWEEN "{}" AND "{}"
    '''
#         AND data_inicio <= "{}" AND data_fim >= "{}"
    start, end = at - between / 2, at + between / 2
    query = waze_query.format(select, str(at), str(at))
    if len(where): query = ' '.join([query, 'AND', where_clause(where)])
    history = bigquery_request(query)
    if len(history):
        if not as_datetime:
            for col in ['ts']:
                if col in history.columns:
                    history[col] = history[col].map(lambda stamp: stamp.strftime('%Y-%m-%d %X'))
        return history
    else:
        return pd.DataFrame(columns=fields_bq)


In [15]:
data = get_waze_bq_history(
    start=dt(2023,1,1), end='now',
    where={}, select='*', as_datetime=False,
)

Forbidden: 403 Access Denied: Table waze-public-dataset:partner_EscritriodeDadosdoRiodeJaneiro.view_alerts_clustered: User does not have permission to query table waze-public-dataset:partner_EscritriodeDadosdoRiodeJaneiro.view_alerts_clustered, or perhaps it does not exist in location US.

Location: US
Job ID: 9c41ec78-8628-4ca3-8a7d-491355447929


In [ ]:
data